In [1]:
import torch

### **Essence of Tensors**

Unlike list or tuple in python, tensors are stored as contiguous memory blocks. Each element in tensor by default is stored as 32-bit (4 Bytes) float.

Instead of seperate memory blocks to store a set of variable. Tensors uses views to access a specific set of indexing.

In [30]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.shape

torch.Size([3, 2])

In [31]:
some_list = list(range(6))
some_list

[0, 1, 2, 3, 4, 5]

In [32]:
some_list[::2] # Start:End:Incrementor

[0, 2, 4]

### **Named Tensors**

As data is transformed through multiple tensors, keeping track of which dimension contains what data can be error-prone.

In [33]:
img_t = torch.randn(3,5,5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [36]:
batch_t = torch.randn(2, 3, 5, 5) 
# shape [batch, channels, rows, columns]

So sometimes the RGB channels are in dimension 0, and sometimes they are in dimension 1. But we can generalize by counting from the end: they are always in dimension `–3`, the third from the end.

In [49]:
img_t.mean(-3)

tensor([[ 0.0399, -0.0449, -0.4924,  0.7555, -0.6933],
        [-0.0703,  0.2722, -0.5207,  0.2151, -0.0496],
        [ 0.1879, -0.4988, -0.1784, -0.6670,  0.3736],
        [-0.0618, -0.4370, -0.8501,  0.2922, -0.0737],
        [ 0.6080, -0.1626, -0.0762, -1.0157,  0.1051]])

In [69]:
batch_t.mean(-3).shape, weights.shape

(torch.Size([2, 5, 5]), torch.Size([3]))

In [68]:
(batch_t * weights.unsqueeze(-1).unsqueeze(-1)).sum(-3).shape

torch.Size([2, 5, 5])

`unsqueeze()` Returns a new tensor with a dimension of size one inserted at the specified position.


In [74]:
img_t.unsqueeze(-3).shape

torch.Size([3, 1, 5, 5])

In [76]:
weights.shape

torch.Size([3])

In [80]:
weights.unsqueeze(-1).unsqueeze(-1).shape, batch_t.shape

(torch.Size([3, 1, 1]), torch.Size([2, 3, 5, 5]))

In a case to multiply the above two tensors, the multiplication would result in a tensor of shape `(2,3,5,5)` as `(3,1,1)` will be broadcasted to all the columns, rows and channels.

In [81]:
img_gray_weighted_fancy = torch.einsum(
    '...chw,c->...hw', img_t, weights)

In [83]:
img_gray_weighted_fancy.shape

torch.Size([5, 5])

In [84]:
weights_named = torch.tensor(
    [0.2126, 0.7152, 0.0722], names=['channels'])

<ipython-input-84-23d37e0242be>:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:934.)
  weights_named = torch.tensor(


In [85]:
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

When we already have a tensor and want to add names (but not change existing ones), we can call the method refine_names on it. Similar to indexing, the ellipsis `(...)` allows you to leave out any number of dimensions. With the rename sibling method, you can also overwrite or drop (by passing in None ) existing names:

In [87]:
img_named = img_t.refine_names(..., 
                               'channels', 'rows', 'columns')
img_named.shape, img_named.names

(torch.Size([3, 5, 5]), ('channels', 'rows', 'columns'))

The standard Python numeric types can be suboptimal for several reasons:

1. Numbers in python are objects: When we store a floating-point number which is 32 bits, python converts it to python pbject with reference counting. This is known as boxing. ANd can be really inefficient to allocate millions of floating point numbers


2. Lists in Python are meant for sequential collections of objects.


3. The Python interpreter is slow compared to optimized, compiled code

In [95]:
torch.Storage(12)

 0.0
 1.5974802493302915e-43
 7.705245871670389e+31
 7.2147958764451125e+22
 2.522590675557027e-18
 2.5929804969848647e-09
 1.0299355986120862e-11
 7.720500350139048e-10
 1.0503973498998675e-05
 4.0518900734642926e-11
 6.712973004141531e-07
 2.9571086513311325e-18
[torch.FloatStorage of size 12]

### Tensor Storage: Under the hood
Values in tensors are allocated in contiguous chunks of memory managed by `torch.Storage` instances. A storage is a one -dimensional array of numerical data: that is, a contiguous
block of memory containing numbers of a given type, such as float (32 bits representing a floating-point number) or int64 (64 bits representing an integer).

The underlying memory is **allocated only once**, managed by the Storage instance.

In [99]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

# The tensor just knows how to translate a pair of indices 
# into a location in the storage.

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

### Tensor metadata: Size, offset, and stride

1. Size: Indicating how many elements across each dimension the tensor represents.


2. Offset: Offset is the index in the storage corresponding to the first element in the tensor.


3. Stride: Stride is the number of elements in the storage that need to be skipped over to obtain the next element along each dimension.

In [102]:
second_point = points[1]
print(second_point)
second_point.storage_offset()

tensor([5., 3.])


2

In [112]:
second_point.size(), second_point.shape

(torch.Size([2]), torch.Size([2]))

In [104]:
points.stride()

(2, 1)

In [110]:
points[2].storage_offset()

4

In [117]:
torch.ones(4,3).stride()

(3, 1)

Accessing an element `i, j` in a 2D tensor results in accessing the `storage_offset + stride[0] * i + stride[1] * j` element in the storage.

### Contiguous tensors

It’s worth noting that calling `contiguous` will do nothing (and will not hurt performance) if the tensor is already contiguous.

In [125]:
points_t = points.T
points_t, points.storage(), points_t.storage()

(tensor([[4., 5., 2.],
         [1., 3., 1.]]),
  4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.FloatStorage of size 6],
  4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.FloatStorage of size 6])

In [127]:
points_t.is_contiguous(), points_t.stride()

(False, (1, 2))

In [129]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [130]:
points_t_cont.storage(), points_t_cont.stride()

( 4.0
  5.0
  2.0
  1.0
  3.0
  1.0
 [torch.FloatStorage of size 6],
 (3, 1))

HDF5 is a portable, widely supported format for representing serialized multidimensional arrays, organized in a nested key-value dictionary. Python supports HDF5 through the h5py library (www.h5py.org), which accepts and returns data in the form of NumPy arrays

In [137]:
import h5py

f = h5py.File('points.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [138]:
f = h5py.File('points.hdf5', 'r')
dset = f['coords']
dset

<HDF5 dataset "coords": shape (3, 2), type "<f4">

In [139]:
dset[-2:]

array([[5., 3.],
       [2., 1.]], dtype=float32)

The data is not loaded when the file is opened or the dataset is required. Rather, the data stays on disk until we request the second and last rows in the dataset. At that point, h5py accesses those two columns and returns a NumPy array-like object encapsulating that region in that dataset that behaves like a NumPy array and has the same API.

## Exercises

In [143]:
a = torch.tensor(list(range(9)), dtype=torch.float)
a

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8.])

In [144]:
a.storage_offset(), a.size(), a.stride()

(0, torch.Size([9]), (1,))

In [146]:
b = a.view(3,3)
b

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [148]:
b.storage(), a.storage()

( 0.0
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
 [torch.FloatStorage of size 9],
  0.0
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
 [torch.FloatStorage of size 9])

In [149]:
c = b[1:, 1:]
# size = [2,2] ; offset = 4 ; stride = (2,1)
c.size(), c.storage_offset(), c.stride()

(torch.Size([2, 2]), 4, (3, 1))

In [154]:
torch.cos(b), b.shape

(tensor([[ 1.0000,  0.5403, -0.4161],
         [-0.9900, -0.6536,  0.2837],
         [ 0.9602,  0.7539, -0.1455]]),
 torch.Size([3, 3]))

In [161]:
b, b.unsqueeze(-1).squeeze(-1)

(tensor([[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]]),
 tensor([[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]]))

In [163]:
a.unsqueeze(-1)

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.]])